In [1]:
# DF를 만들든, data를 불러오든 해서, 원하는 pivot 형태로 만드는 게 가장 이상적일 수도 있음
# groupby, 집계함수, agg로 골라서, 원하는 컬럼을 뽑아서 DF를 만드는 것
# => 하고자 하는 분석에 따라 정리해서 DF을 별도로 할 수 있는 것

In [2]:
import pandas as pd
import seaborn

In [3]:
# Data 로딩
df = seaborn.load_dataset('titanic')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.7+ KB


In [8]:
# 결측치 확인
df.isna().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [14]:
# 성별, 등석, 나이에 따른 생존율
# 일단, 쓰고자 하는 컬럼들만 뽑아서 별도의 DF를 만들어보자
df2 = pd.concat([df['sex'],df['pclass'],df['age'],df['survived']],axis=1) 
df2

,sex,pclass,age,survived
0,male,3,22.0,0
1,female,1,38.0,1
2,female,3,26.0,1
3,female,1,35.0,1
4,male,3,35.0,0
...,...,...,...,...
886,male,2,27.0,0
887,female,1,19.0,1
888,female,3,NaN,0
889,male,1,26.0,1


### age -> 결측치 처리

In [20]:
# 결측치 값을 평균값으로 넣어주도록 해보자
# 그런데, age컬럼 자체의 나이로 평균을 처리하면, 생존여부에 따른 것들이 다르므로 그룹화로 구분을 해 주어야 함

In [21]:
# survived와 성별에 따라 그룹화(groupby) -> 그 그룹별로 평균값을 찾아야 함
print(f"----- 그룹화o : {df.groupby(['survived','sex']).age.mean()}---",
      f"----- 그룹화x : {df.age.mean()}", sep='\n\n')

----- 그룹화o : survived  sex   
0         female    25.046875
          male      31.618056
1         female    28.847716
          male      27.276022
Name: age, dtype: float64---

----- 그룹화x : 29.69911764705882


In [40]:
# 그룹별 평균을 결측치로 채워 넣으면서 새로운 시리즈를 만들어서 => age_02 컬럼을 추가
age_02 = df2.groupby(['survived','sex']).age.transform(lambda g: g.fillna(g.mean()))
type(age_02)

pandas.core.series.Series

In [29]:
df2.columns

Index(['sex', 'pclass', 'age', 'survived', 'age_02'], dtype='object')

In [43]:
# df2.insert(3, 'age_02', age_02)
df2

,sex,pclass,age,age_02,survived
0,male,3,22.0,22.000000,0
1,female,1,38.0,38.000000,1
2,female,3,26.0,26.000000,1
3,female,1,35.0,35.000000,1
4,male,3,35.0,35.000000,0
...,...,...,...,...,...
886,male,2,27.0,27.000000,0
887,female,1,19.0,19.000000,1
888,female,3,NaN,25.046875,0
889,male,1,26.0,26.000000,1


### age -> 연령대 범위로 나누기

In [109]:
# Age_02의 최대값이 80임을 확인
df2['age_02'].max()

# Age_02를 범주형으로 나눠보자 
age_02_label = pd.cut(x = df2['age_02'], bins=4,
                                  labels = ['~10대','~30대','~50대','50대~'], include_lowest = True)

### pclass

In [ ]:
# pclass의 카테고리별 라벨을 적어줘서 처리해줘

In [79]:
df['pclass'].unique()

array([3, 1, 2], dtype=int64)

In [80]:
df['pclass'].value_counts()
# 1:2:3 = 가:나:다 => 뭘로 구분해야되징

3    491
1    216
2    184
Name: pclass, dtype: int64

In [81]:
df.columns

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')

In [83]:
# Pclass 컬럼 라벨 데이터 생성
pclassLabel = df['pclass'].replace({1:'가', 2:'나', 3:'다'})

# DF에 컬럼 추가
df2.insert(2, 'pclass_label', pclassLabel)

In [84]:
df2

,sex,pclass,pclass_label,age,age_02,survived,alive
0,male,3,다,22.0,22.000000,0,no
1,female,1,가,38.0,38.000000,1,yes
2,female,3,다,26.0,26.000000,1,yes
3,female,1,가,35.0,35.000000,1,yes
4,male,3,다,35.0,35.000000,0,no
...,...,...,...,...,...,...,...
886,male,2,나,27.0,27.000000,0,no
887,female,1,가,19.0,19.000000,1,yes
888,female,3,다,NaN,25.046875,0,no
889,male,1,가,26.0,26.000000,1,yes


In [85]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sex           891 non-null    object 
 1   pclass        891 non-null    int64  
 2   pclass_label  891 non-null    object 
 3   age           714 non-null    float64
 4   age_02        891 non-null    float64
 5   survived      891 non-null    int64  
 6   alive         891 non-null    object 
dtypes: float64(2), int64(2), object(3)
memory usage: 48.9+ KB


### 형변환

- 일단 구해보고 안되는 거라면 바꿔주도록 해보자

In [ ]:
# 성별, 등석, 나이에 따른 생존율

In [103]:
# cut으로 나이 연령대를 나누고
# survived 든 alive 든 

# 각 요소별
all = df2['survived'].count()
alive_sum = df2['survived'].sum()
alive_pct = b / a * 100
alive_pct

38.38383838383838

In [100]:
# 승선 인원 중 여성의 숫자
survived_female = df2['sex'][df2['sex']=='female'].count()
# 승선 인원 중 남성의 숫자
survived_male = df2['sex'][df2['sex']=='male'].count()

314

In [ ]:
# 연령대 별로 나누고, 연령대별로 수치를 더하고, 숫자들 변수들 넣어서 나눠서 생존율을 구해봐

In [ ]:
# => 전처리를 한 df2의 그룹화(성별, 등석, 나이) / 평균(.mean()) / 

In [ ]:
s